In [16]:
#1
import pandas as pd
import numpy as np
import json

import psycopg2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import xgboost as xgb

from sklearn import metrics
from xgboost import plot_importance
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor   
import seaborn as sns
%matplotlib inline
import pickle
import gc
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
plt.style.use('bmh')
from pandas import Series, DataFrame
from sklearn.model_selection import GridSearchCV


import traceback
import sys
import re
import datetime
datetime.timedelta()

datetime.timedelta(0)

In [17]:
def risk_seg(pred):
    if pred<=1000:
        return 'low_priority'
    elif pred<=5000:
        return 'medium_priority'
    else:
        return 'high_priority'

In [18]:
def func1(risk_segment,allocation,existing_comms,paid_file,S_AMT):
    print(risk_segment.shape)
    
    #risk_segment.shape
    S_AMT['Loan id']=S_AMT['Loan id'].astype(int).astype(str)
    risk_segment['AgreementNumber']=risk_segment['AgreementNumber'].astype(int).astype(str)
    #june22_paid['Loan_ID']=june22_paid['Loan_ID'].astype(int).astype(str)
    risk_segment=pd.merge(risk_segment,S_AMT,how='left',left_on='AgreementNumber',right_on='Loan id')
    risk_segment.shape
    
    
    risk_segment.loc[:, 'risk_priority'] = risk_segment['S_AMT'].apply(lambda x: risk_seg(x))
    risk_segment['risk_priority'].value_counts()
    
    
    #existing_comms['Loan_ID']=existing_comms['Loan_ID'].astype(int).astype(str)
    
    existing_comms['Call_Start_Time'] = pd.to_datetime(existing_comms['Call_Start_Time'], errors='coerce')
    existing_comms['Call_End_Time'] = pd.to_datetime(existing_comms['Call_End_Time'], errors='coerce')
    existing_comms['call_duration'] = (existing_comms.Call_End_Time-existing_comms.Call_Start_Time).dt.total_seconds()

    def process_call_duration(row):
        if row['Dialer_Disposition'] in ['Answered', 'Agent_Answered']:
            return row['call_duration']
        else:
            return 0

    existing_comms['call_duration'] = existing_comms.apply(lambda x: process_call_duration(x), axis=1)


    existing_comms['call_picked']=existing_comms['call_duration'].apply(lambda x: 1 if x>0 else 0)
    existing_comms['call_responsive']=existing_comms['call_duration'].apply(lambda x: 1 if x>=30 else 0)

    ## Aggregating the data
    existing_comms_v2=existing_comms.groupby('Loan_ID').agg(calls_count=('Contact No ','count'),
                                             total_call_duration=('call_duration','sum'),
                                             mean_call_duration=('call_duration','mean'),
                                             median_call_duration=('call_duration','median'),
                                             total_call_picked=('call_picked','sum'),
                                             total_call_responsive=('call_responsive','sum')
                                            # paid_flag=('paid_flag','first')
                                             ).reset_index()



    existing_comms_v2['call_picked_%']=existing_comms_v2['total_call_picked']/existing_comms_v2['calls_count']*100
    existing_comms_v2['call_response_%']=existing_comms_v2['total_call_responsive']/existing_comms_v2['calls_count']*100

    existing_comms_v2['call_picked_flag']=existing_comms_v2['total_call_picked'].apply(lambda x: 1 if x>0 else 0)
    
    
    risk_segment['AgreementNumber']=risk_segment['AgreementNumber'].astype(int).astype(str)
    #existing_comms_v2['Loan_ID']=existing_comms_v2['Loan_ID'].astype(int).astype(str)
    #existing_comms_v2['Loan_ID']=existing_comms_v2['Loan_ID'].astype(int).astype(str)
    #Jan22_raw['AgreementNumber']=Jan22_raw['AgreementNumber'].astype(int).astype(str)
    risk_segment_picked=pd.merge(risk_segment,existing_comms_v2[['Loan_ID','call_picked_flag']],how='left',left_on='AgreementNumber',right_on='Loan_ID')
    risk_segment_picked_v2=pd.merge(risk_segment,existing_comms_v2,how='left',left_on='AgreementNumber',right_on='Loan_ID')

    risk_segment_picked=risk_segment_picked.drop(['OS', 'Loan_ID_x','Loan_ID_y', 'Loan id','Loan_ID'],axis=1)
    risk_segment_picked_v2=risk_segment_picked_v2.drop(['OS', 'Loan_ID_x','Loan_ID_y', 'Loan id','Loan_ID'],axis=1)
    
    return risk_segment_picked,risk_segment_picked_v2

    

In [19]:
def func2(risk_segment,allocation,existing_comms,paid_file):
    
    ## converting loan ID to numeric datatype
    
    existing_comms=existing_comms[existing_comms['Loan_ID'].isin(allocation['AgreementNumber'])]
    existing_comms['Loan_ID']=pd.to_numeric(existing_comms['Loan_ID'], errors='coerce')

    
    existing_comms['Call_Start_Time'] = pd.to_datetime(existing_comms['Call_Start_Time'], errors='coerce')
    existing_comms['Call_End_Time'] = pd.to_datetime(existing_comms['Call_End_Time'], errors='coerce')
    existing_comms['call_duration'] = (existing_comms.Call_End_Time-existing_comms.Call_Start_Time).dt.total_seconds()

    def process_call_duration(row):
        if row['Dialer_Disposition'] in ['Answered', 'Agent_Answered']:
            return row['call_duration']
        else:
            return 0

    existing_comms['call_duration'] = existing_comms.apply(lambda x: process_call_duration(x), axis=1)


    existing_comms['call_picked']=existing_comms['call_duration'].apply(lambda x: 1 if x>0 else 0)
    existing_comms['call_responsive']=existing_comms['call_duration'].apply(lambda x: 1 if x>=30 else 0)

    ## Aggregating the data
    existing_comms_v2=existing_comms.groupby('Loan_ID').agg(calls_count=('Contact No ','count'),
                                             total_call_duration=('call_duration','sum'),
                                             mean_call_duration=('call_duration','mean'),
                                             median_call_duration=('call_duration','median'),
                                             total_call_picked=('call_picked','sum'),
                                             total_call_responsive=('call_responsive','sum')
                                            # paid_flag=('paid_flag','first')
                                             ).reset_index()

    existing_comms_v2['call_picked_%']=existing_comms_v2['total_call_picked']/existing_comms_v2['calls_count']*100
    existing_comms_v2['call_response_%']=existing_comms_v2['total_call_responsive']/existing_comms_v2['calls_count']*100
    
    df_picked=existing_comms_v2.copy()
    
    df_picked.shape
    
    #Changing Dates
    paid_file.shape
    paid_file['Date Repaid'] = pd.to_datetime(paid_file['Date Repaid'], errors='coerce')
    paid_file['day']=paid_file['Date Repaid'].dt.day
    paid_file.shape
    #paid_file=june22_paid[(paid_file['day']>=0) & (june22_paid['day']<=31)]
    paid_file.shape

    #june22_paid.drop(['paid_flag'],axis=1,inplace=True)
    paid_file['paid']=1
    paid_file=paid_file.rename(columns={'AgreementNumber':'Loan_ID','paid':'paid_flag'})
    paid_file.drop_duplicates(subset ="Loan_ID", keep = 'first', inplace = True)
    paid_file.shape


    paid_file['Loan_ID']=paid_file['Loan_ID'].astype(int).astype(str)
    #S_AMT['Loan id']=S_AMT['Loan id'].astype(int).astype(str)

#     paid_file=pd.merge(paid_file ,S_AMT,how='left',left_on='Loan_ID',right_on='Loan id')
#     paid_file.shape

    allocation['AgreementNumber']=allocation['AgreementNumber'].astype(int).astype(str)
    df_picked['Loan_ID']=df_picked['Loan_ID'].astype(int).astype(str)

    df_picked=pd.merge(allocation,df_picked[['Loan_ID','total_call_picked','calls_count','total_call_duration']],how='left',left_on='AgreementNumber',right_on='Loan_ID')
    df_picked.shape
    df_picked=pd.merge(df_picked,paid_file[['Loan_ID','paid_flag','Repayment']],how='left',left_on='AgreementNumber',right_on='Loan_ID')
    df_picked.shape


    df_picked['paid_flag']=df_picked['paid_flag'].fillna(0)
    df_picked['picked_flag']=df_picked['total_call_picked'].apply(lambda x : 1 if x>0 else 0)
    
    kk=df_picked.groupby('picked_flag').agg(comms_count=('calls_count','sum'),cust_count=('AgreementNumber','count'),paid_count=('paid_flag','sum'),paid_amount=('Repayment','sum'),total_time_spent=('total_call_duration','sum')).reset_index()

    kk['churn_per_paid_cust']=round(kk['comms_count']/kk['paid_count'],1)
    kk['paid_dist']=round(kk['paid_count']/sum(kk['paid_count'])*100,2)
    kk['paid_rate']=round(kk['paid_count']/kk['cust_count']*100,2)
    
    return kk


In [38]:
def func3(risk_segment_file_to_internal_picked,tag):
    print(tag[0])
    print(tag[1])
    print(tag[2])
    print(tag[3])
    
    risk_segment_order=['low_risk','medium_risk','high_risk']
    risk_priority_order=['low_priority','medium_priority','high_priority']
    
    val_pred_par = pd.crosstab(risk_segment_file_to_internal_picked.risk_segment, risk_segment_file_to_internal_picked.risk_priority,values=risk_segment_file_to_internal_picked['calls_count'],aggfunc=np.sum,
                   normalize=False, margins=True)
    val_pred_par1 = pd.crosstab(risk_segment_file_to_internal_picked.risk_segment, risk_segment_file_to_internal_picked.risk_priority,values=risk_segment_file_to_internal_picked['flag'],aggfunc=np.sum, normalize=False, margins=True)
    val_pred_par2 = pd.crosstab(risk_segment_file_to_internal_picked.risk_segment, risk_segment_file_to_internal_picked.risk_priority,values=risk_segment_file_to_internal_picked['paid_flag'],aggfunc=np.sum, normalize=False, margins=True)
    val_pred_par3 = pd.crosstab(risk_segment_file_to_internal_picked.risk_segment, risk_segment_file_to_internal_picked.risk_priority,values=risk_segment_file_to_internal_picked['S_AMT'],aggfunc=np.sum, normalize=False, margins=True)
    
    fig = plt.figure(figsize=(20, 10))

    ax1 = fig.add_subplot(221)
    sns.heatmap(val_pred_par, 
        annot=True,
        linewidths=0.3,
        robust=True,
        cmap=sns.light_palette("seagreen", as_cmap=True),
        vmin=0, vmax=7, 
    #     center=0.5,
        annot_kws={'size': 14},
        ax=ax1, fmt='.10g'
    )
    ax1.set_xticklabels(
        ax1.get_xticklabels(),
        rotation=0,
        horizontalalignment='center'
    );
    
   
    ax2 = fig.add_subplot(222)
    sns.heatmap(val_pred_par1, 
        annot=True,
        linewidths=0.3,
        robust=True,
        cmap=sns.light_palette("seagreen", as_cmap=True),
        vmin=0, vmax=7, 
    #     center=0.5,
        annot_kws={'size': 14},
        ax=ax2, fmt='.10g'
    )
    ax2.set_xticklabels(
        ax2.get_xticklabels(),
        rotation=0,
        horizontalalignment='center'
    );
   
    ax3 = fig.add_subplot(223)
    sns.heatmap(val_pred_par2, 
        annot=True,
        linewidths=0.3,
        robust=True,
        cmap=sns.light_palette("seagreen", as_cmap=True),
        vmin=0, vmax=7, 
    #     center=0.5,
        annot_kws={'size': 14},
        ax=ax3, fmt='.10g'
    )
    ax3.set_xticklabels(
        ax3.get_xticklabels(),
        rotation=0,
        horizontalalignment='center'
    );
   
    ax4 = fig.add_subplot(224)
    sns.heatmap(val_pred_par3, 
        annot=True,
        linewidths=0.3,
        robust=True,
        cmap=sns.light_palette("seagreen", as_cmap=True),
        vmin=0, vmax=7, 
    #     center=0.5,
        annot_kws={'size': 14},
        ax=ax4, fmt='.10g'
    )
    ax4.set_xticklabels(
        ax4.get_xticklabels(),
        rotation=0,
        horizontalalignment='center'
    );
    plt.show()


In [ ]:
# def func4(risk_segment,allocation,existing_comms,paid_file):
    